# Collision Avoidance - Data Collection

If you ran through the Day 1 notebook, hopefully you're enjoying how easy it can be to make your Jetbot move around using functions, widgets and even a controller! Thats very cool!  But what's even cooler, is making your Jetbot move around all by itself! 

The goal of the Collision Avodaince task is to allow the Jetbot to move around freely until it comes across a "dangerous situation" then it will turn around (avoiding collision) and continue on moving freely.

To do this, Firstly, we'll manually place the robot in scenarios where it's "safety bubble" is violated, and label these scenarios as ``blocked``.  We save a snapshot of what the robot sees along with this label.

Second, we'll manually place the robot in scenarios where it's safe to move forward, and label these scenarios as ``free``.  Likewise, we save a snapshot along with this label.

That's all that we'll be doing in this notebook, "**data collection**".  Once we have collected lots of images for the 2 classes, we'll upload this data to a GPU enabled machine where we'll *train* a neural network to predict whether the robot's safety bubble is being violated based off of the image it sees.  We'll use this to implement a simple collision avoidance behavior in the end :)

## Display live camera feed

So let's get started.  First, let's initialize and display our camera like we did in the *Remote_control* notebook in Day 1. 

> Our neural network takes a 224x224 pixel image as input.  We'll set our camera to that size to minimize the filesize of our dataset.
> In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image_widget = widgets.Image(format='jpeg', width=224, height=224) 

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Awesome, next let's create a few directories where we will use to store all our data.  We'll create a folder named ``dataset`` that will contain two sub-folders ``free`` and ``blocked``, where we will place the images for each scenario.

In [ ]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

If you refresh the Jupyter file browser on the left, you should now see those directories appear.  Next, let's create and display some buttons which we'll use to save snapshots for each class label.  We'll also add some text boxes that will display how many images of each category that we've collected so far. This is useful because we want to make
sure we collect about as many ``free`` images as ``blocked`` images.  It also helps to know how many images we've collected overall.

In [ ]:
import ipywidgets as widgets

button_layout = widgets.Layout(width='128px', height='64px')

free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)

free_count = widgets.IntText(layout=button_layout)
blocked_count = widgets.IntText(layout=button_layout)

display(widgets.HBox([free_button, free_count]))
display(widgets.HBox([blocked_button, blocked_count]))

Right now, these buttons wont do anything.  We have to attach functions to save images for each category to the buttons' ``on_click`` event.  We'll save the value
of the ``Image`` widget (rather than the camera), because it's already in compressed JPEG format!

To make sure we don't repeat any file names (even across different machines!) we'll use the ``uuid`` package in python, which defines the ``uuid1`` method to generate
a unique identifier.  This unique identifier is generated from information like the current time and the machine address.

In [ ]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

Great! now lets display our camera image together with our buttons which we just created so that we can have a better view of what our Jetbot is seeing.

In [ ]:
display(image)
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

Now let's go ahead and collect some data!

### Task 1: Collect Image Classification Data   
- Move around and collect atleast 100 images for each class ``blocked`` and ``free``. 
- For the ``Blocked`` class capture images of objects that you observe in the room such as walls, objects, etc.
- For the ``Free`` class capture scenarios where the Jetbot is safe to move around in such as floors, carpets, etc.
- Make sure to capture images using varying orientations and different lighting conditions.

## Save dataset in a zip (optional)

Once you've collected enough data, we can choose to copy that data to a GPU desktop or cloud machine for training. To do that, we can call the following *terminal* command to compress
our dataset folder into a single *zip* file.

> The ! prefix indicates that we want to run the cell as a *shell* (or *terminal*) command.

> The -r flag in the zip command below indicates *recursive* so that we include all nested files, the -q flag indicates *quiet* so that the zip command doesn't print any output

In [ ]:
!zip -r -q dataset.zip dataset

You should see a file named ``dataset.zip`` in the Jupyter Lab file browser.

Next, Let's move on to the "2-Train_model.ipynb" notebook so that we can start training our model on the collected dataset!